<a href="https://colab.research.google.com/github/jstephens/tideprediction/blob/main/Tide_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries


In [ ]:
!pip install keras --quiet
!pip install tensorflow --quiet
!pip install innvestigate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 2.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import innvestigate
from bokeh.io import show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import DatetimeTickFormatter, DatetimeAxis
from bokeh.models import HoverTool
from bokeh.models import FixedTicker
from bokeh.embed import components
from bokeh.resources import CDN
from bokeh.embed import autoload_static
import os
from getpass import getpass
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Conv2D, MaxPooling2D, Bidirectional, InputLayer, Concatenate, Dropout, Input, GRU
from sklearn.model_selection import GridSearchCV
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
import imageio
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.pyplot as plt
import xgboost as xgb
import datetime

MessageError: ignored

# Data Load and Initial Transformation

In [ ]:
X_train = np.load('drive/MyDrive/source/X_train_surge_new.npz')
X_test = np.load('drive/MyDrive/source/X_test_surge_new.npz')
Y_train = pd.read_csv('drive/MyDrive/source/Y_train_surge.csv',index_col = 'id_sequence')
Y_test = pd.read_csv('drive/MyDrive/source/Y_train_surge.csv',index_col = 'id_sequence')

df_X_test = pd.DataFrame.from_dict({item: X_test[item] for item in X_test.files}, orient='index')
df_X_test = df_X_test.T
df_X_test = df_X_test.set_index('id_sequence')

df_X_train = pd.DataFrame.from_dict({item: X_train[item] for item in X_train.files}, orient='index')
df_X_train = df_X_train.T
df_X_train = df_X_train.set_index('id_sequence')

In [ ]:
Y_train

In [ ]:
def timestamp_to_datetime(timestamp):
    return pd.to_datetime('1970-01-01') + pd.to_timedelta(timestamp, unit='s')

In [ ]:
df_X_train['t_slp'] = df_X_train['t_slp'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_train['t_surge1_input'] = df_X_train['t_surge1_input'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_train['t_surge2_input'] = df_X_train['t_surge2_input'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_train['t_surge1_output'] = df_X_train['t_surge1_output'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))
df_X_train['t_surge2_output'] = df_X_train['t_surge2_output'].apply(lambda x: np.array([timestamp_to_datetime(t) for t in x]))

# Visualization


In [ ]:
df_plotly = df_X_train.copy()

df_plotly_row_100 = pd.DataFrame(df_plotly.iloc[100]).T
df_plotly_row_101 = pd.DataFrame(df_plotly.iloc[101]).T
df_plotly_row_102 = pd.DataFrame(df_plotly.iloc[102]).T

# row 100
t_slp_df_row_100 = pd.DataFrame(df_plotly_row_100['t_slp'])
t_surge1_input_df_row_100 = pd.DataFrame(df_plotly_row_100['t_surge1_input'])
t_surge2_input_df_row_100 = pd.DataFrame(df_plotly_row_100['t_surge2_input'])
t_surge1_output_df_row_100 = pd.DataFrame(df_plotly_row_100['t_surge1_output'])
t_surge2_output_df_row_100 = pd.DataFrame(df_plotly_row_100['t_surge2_output'])

t_slp_df_row_100 = t_slp_df_row_100.explode('t_slp').reset_index(drop=True)
t_surge1_input_df_row_100 = t_surge1_input_df_row_100.explode('t_surge1_input').reset_index(drop=True)
t_surge2_input_df_row_100 = t_surge2_input_df_row_100.explode('t_surge2_input').reset_index(drop=True)
t_surge1_output_df_row_100 = t_surge1_output_df_row_100.explode('t_surge1_output').reset_index(drop=True)
t_surge2_output_df_row_100 = t_surge2_output_df_row_100.explode('t_surge2_output').reset_index(drop=True)

# row 101
t_slp_df_row_101 = pd.DataFrame(df_plotly_row_101['t_slp'])
t_surge1_input_df_row_101 = pd.DataFrame(df_plotly_row_101['t_surge1_input'])
t_surge2_input_df_row_101 = pd.DataFrame(df_plotly_row_101['t_surge2_input'])
t_surge1_output_df_row_101 = pd.DataFrame(df_plotly_row_101['t_surge1_output'])
t_surge2_output_df_row_101 = pd.DataFrame(df_plotly_row_101['t_surge2_output'])

t_slp_df_row_101 = t_slp_df_row_101.explode('t_slp').reset_index(drop=True)
t_surge1_input_df_row_101 = t_surge1_input_df_row_101.explode('t_surge1_input').reset_index(drop=True)
t_surge2_input_df_row_101 = t_surge2_input_df_row_101.explode('t_surge2_input').reset_index(drop=True)
t_surge1_output_df_row_101 = t_surge1_output_df_row_101.explode('t_surge1_output').reset_index(drop=True)
t_surge2_output_df_row_101 = t_surge2_output_df_row_101.explode('t_surge2_output').reset_index(drop=True)

# row 102
t_slp_df_row_102 = pd.DataFrame(df_plotly_row_102['t_slp'])
t_surge1_input_df_row_102 = pd.DataFrame(df_plotly_row_102['t_surge1_input'])
t_surge2_input_df_row_102 = pd.DataFrame(df_plotly_row_102['t_surge2_input'])
t_surge1_output_df_row_102 = pd.DataFrame(df_plotly_row_102['t_surge1_output'])
t_surge2_output_df_row_102 = pd.DataFrame(df_plotly_row_102['t_surge2_output'])

t_slp_df_row_102 = t_slp_df_row_102.explode('t_slp').reset_index(drop=True)
t_surge1_input_df_row_102 = t_surge1_input_df_row_102.explode('t_surge1_input').reset_index(drop=True)
t_surge2_input_df_row_102 = t_surge2_input_df_row_102.explode('t_surge2_input').reset_index(drop=True)
t_surge1_output_df_row_102 = t_surge1_output_df_row_102.explode('t_surge1_output').reset_index(drop=True)
t_surge2_output_df_row_102 = t_surge2_output_df_row_102.explode('t_surge2_output').reset_index(drop=True)

In [ ]:
colorvar = '#76878f'

x = 15
source_row100 = ColumnDataSource(data=dict(x=pd.to_datetime(t_slp_df_row_100['t_slp']), y=[x] * len(t_slp_df_row_100), size=[10] * len(t_slp_df_row_100),color=[colorvar] * len(t_slp_df_row_100)))
source2_row100 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge1_input_df_row_100['t_surge1_input']), y=[x+2] * len(t_surge1_input_df_row_100), size=[10] * len(t_surge1_input_df_row_100),color=[colorvar] * len(t_surge1_input_df_row_100)))
source3_row100 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge2_input_df_row_100['t_surge2_input']), y=[x+3] * len(t_surge2_input_df_row_100), size=[10] * len(t_surge2_input_df_row_100),color=[colorvar] * len(t_surge2_input_df_row_100)))
source2_pred_row100 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge1_output_df_row_100['t_surge1_output']), y=[x+2] * len(t_surge1_output_df_row_100), size=[10] * len(t_surge1_output_df_row_100),color=['lightgray'] * len(t_surge2_output_df_row_100)))
source3_pred_row100 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge2_output_df_row_100['t_surge2_output']), y=[x+3] * len(t_surge2_output_df_row_100), size=[10] * len(t_surge2_output_df_row_100),color=['lightgray'] * len(t_surge2_output_df_row_100)))

x2 = 8
source_row101 = ColumnDataSource(data=dict(x=pd.to_datetime(t_slp_df_row_101['t_slp']), y=[x2] * len(t_slp_df_row_101), size=[10] * len(t_slp_df_row_101),color=[colorvar] * len(t_slp_df_row_101)))
source2_row101 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge1_input_df_row_101['t_surge1_input']), y=[x2+2] * len(t_surge1_input_df_row_101), size=[10] * len(t_surge1_input_df_row_101),color=[colorvar] * len(t_surge1_input_df_row_101)))
source3_row101 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge2_input_df_row_101['t_surge2_input']), y=[x2+3] * len(t_surge2_input_df_row_101), size=[10] * len(t_surge2_input_df_row_101),color=[colorvar] * len(t_surge2_input_df_row_101)))
source2_pred_row101 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge1_output_df_row_101['t_surge1_output']), y=[x2+2] * len(t_surge1_output_df_row_101), size=[10] * len(t_surge1_output_df_row_101),color=['lightgray'] * len(t_surge2_output_df_row_101)))
source3_pred_row101 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge2_output_df_row_101['t_surge2_output']), y=[x2+3] * len(t_surge2_output_df_row_101), size=[10] * len(t_surge2_output_df_row_101),color=['lightgray'] * len(t_surge2_output_df_row_101)))

x3 = 1
source_row102 = ColumnDataSource(data=dict(x=pd.to_datetime(t_slp_df_row_102['t_slp']), y=[x3] * len(t_slp_df_row_102), size=[10] * len(t_slp_df_row_102),color=[colorvar] * len(t_slp_df_row_102)))
source2_row102 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge1_input_df_row_102['t_surge1_input']), y=[x3+2] * len(t_surge1_input_df_row_102), size=[10] * len(t_surge1_input_df_row_102),color=[colorvar] * len(t_surge1_input_df_row_102)))
source3_row102 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge2_input_df_row_102['t_surge2_input']), y=[x3+3] * len(t_surge2_input_df_row_102), size=[10] * len(t_surge2_input_df_row_102),color=[colorvar] * len(t_surge2_input_df_row_102)))
source2_pred_row102 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge1_output_df_row_102['t_surge1_output']), y=[x3+2] * len(t_surge1_output_df_row_102), size=[10] * len(t_surge1_output_df_row_102),color=['lightgray'] * len(t_surge2_output_df_row_102)))
source3_pred_row102 = ColumnDataSource(data=dict(x=pd.to_datetime(t_surge2_output_df_row_102['t_surge2_output']), y=[x3+3] * len(t_surge2_output_df_row_102), size=[10] * len(t_surge2_output_df_row_102),color=['lightgray'] * len(t_surge2_output_df_row_102)))

p = figure(plot_width=800, plot_height=400, x_axis_label='Timestamp', y_axis_label='', toolbar_location=None, x_axis_type='datetime')

y_ticks = [1,2.5,3,4,8,9.5,10,11,15,16.5,17,18]
y_labels = ['Encoded Image Recorded','Row 102                                                               ','Surge Time, City 1','Surge Time, City 2',
            'Encoded Image Recorded','Row 101                                                               ','Surge Time, City 1','Surge Time, City 2',
            'Encoded Image Recorded','Row 100                                                               ','Surge Time, City 1','Surge Time, City 2']
p.yaxis.ticker = FixedTicker(ticks=y_ticks)
p.yaxis.major_label_overrides = {tick: label for tick, label in zip(y_ticks, y_labels)}

p.ygrid.grid_line_color = 'white'

sources = [
    source_row100, source2_row100, source3_row100, source2_pred_row100, source3_pred_row100,
    source_row101, source2_row101, source3_row101, source2_pred_row101, source3_pred_row101,
    source_row102, source2_row102, source3_row102, source2_pred_row102, source3_pred_row102
]

for source in sources:
    p.circle('x', 'y', source=source, size='size', alpha=0.5,color='color')

p.x_range=Range1d(start=pd.Timestamp('1950-11-06 12:30:00'), end=pd.Timestamp('1950-11-18 12:30:00'))

p.y_range=Range1d(start=0, end=19)

p.xaxis.major_tick_line_color = None
p.xaxis.minor_tick_line_color = None
p.yaxis.major_tick_line_color = None
p.yaxis.minor_tick_line_color = None

hover = HoverTool(
    tooltips=[
        ("Time", "@x{%F %T}")
    ],
    formatters={"@x": "datetime"},
    mode='mouse'
)
p.add_tools(hover)
output_notebook()
show(p)

The plot above represents three typical rows for the X training dataset. Each row represents 5 days but will overlap with other rows temporally. 

In [ ]:
script, div = components(p)
print(div)
print(script)

<div class="bk-root" id="887cd2a0-ef04-4ea0-9c6f-49cf587d5834" data-root-id="1017"></div>
    <script type="text/javascript">
        (function() {
  const fn = function() {
    Bokeh.safely(function() {
      (function(root) {
        function embed_document(root) {
        const docs_json = '{"994e1a88-53fd-44fa-995a-fcd24e8633e0":{"defs":[],"roots":{"references":[{"attributes":{},"id":"1164","type":"Selection"},{"attributes":{},"id":"1163","type":"UnionRenderers"},{"attributes":{},"id":"1165","type":"UnionRenderers"},{"attributes":{"data":{"color":["#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f","#76878f"],"size":[10,10,10,10,10,10,10,10,10,10,10,

In [ ]:
# Retrieve the 40 images from the row of interest
images = df_X_train['slp'][5599]

# Create an empty list to store the image file names
image_files = []

# Loop over each image and save it as a file
for i in range(len(images)):
    # Create the file name with a 4-digit index
    filename = f'image_{i:04d}.png'
    
    # Save the image as a file
    plt.imsave(filename, images[i], cmap='gray')
    
    # Add the file name to the list
    image_files.append(filename)

# Use imageio to create the gif
with imageio.get_writer('encodedimage.gif', mode='I') as writer:
    # Loop over each image file and add it to the gif
    for filename in image_files:
        image = imageio.imread(filename)
        writer.append_data(image)

<ipython-input-11-4d5f8dacc45c>:22: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


# XGBoost

In [ ]:
def get_nth_day(timestamps):
    first_timestamp = timestamps[0]
    year_start = datetime.datetime(first_timestamp.year, 1, 1)
    return int((first_timestamp - year_start).days + 1)

def get_first_year(timestamps):
    return timestamps[0].year

def extract_hour(timestamps):
    return np.array([pd.Timestamp(ts, unit='s').hour for ts in timestamps])

# apply the functions to each element in the column using pandas' apply() function
df_X_train['date'] = df_X_train['t_slp'].apply(lambda x: get_nth_day(x))
df_X_train['year'] = df_X_train['t_slp'].apply(lambda x: get_first_year(x))
df_X_train['t_slp'] = df_X_train['t_slp'].apply(lambda x: extract_hour(x))
df_X_train['t_surge1_input'] = df_X_train['t_surge1_input'].apply(lambda x: extract_hour(x))
df_X_train['t_surge2_input'] = df_X_train['t_surge2_input'].apply(lambda x: extract_hour(x))
df_X_train['t_surge1_output'] = df_X_train['t_surge1_output'].apply(lambda x: extract_hour(x))
df_X_train['t_surge2_output'] = df_X_train['t_surge2_output'].apply(lambda x: extract_hour(x))

In [ ]:
df_X_train = df_X_train.tail(365*2)

In [ ]:
df_X_train

,t_slp,slp,t_surge1_input,surge1_input,t_surge2_input,surge2_input,t_surge1_output,t_surge2_output,date,year
id_sequence,,,,,,,,,,
4870,"[2, 6, 8, 11, 15, 17, 21, 0, 2, 6, 8, 12, 15, ...","[[[101799.4, 101754.4, 101706.4, 101656.4, 101...","[16, 5, 16, 5, 16, 6, 17, 7, 18, 8]","[-0.16049244, 0.25846964, -0.09010681, -0.1705...","[17, 6, 18, 7, 18, 8, 20, 8, 21, 9]","[0.40714034, 1.0022833, 0.45743412, 0.42390493...","[20, 8, 21, 9, 22, 10, 0, 11, 0, 13]","[23, 10, 0, 11, 0, 13, 1, 14, 2, 15]",204,2004
4871,"[6, 8, 12, 15, 17, 21, 0, 2, 6, 8, 11, 15, 17,...","[[[101773.58, 101729.58, 101691.58, 101658.58,...","[16, 5, 16, 6, 17, 7, 18, 8, 20, 8]","[-0.09010681, -0.17054753, 0.27187642, 0.44281...","[18, 7, 18, 8, 20, 8, 21, 9, 23, 10]","[0.45743412, 0.42390493, -0.0035921182, 0.2646...","[21, 9, 22, 10, 0, 11, 0, 13, 1, 14]","[0, 11, 0, 13, 1, 14, 2, 15, 4, 16]",205,2004
4872,"[6, 8, 11, 15, 17, 21, 0, 2, 6, 8, 12, 15, 17,...","[[[101834.67, 101812.67, 101795.67, 101779.67,...","[16, 6, 17, 7, 18, 8, 20, 8, 21, 9]","[0.27187642, 0.44281295, 0.28863493, 0.6640249...","[18, 8, 20, 8, 21, 9, 23, 10, 0, 11]","[-0.0035921182, 0.2646413, -0.2131495, 0.30655...","[22, 10, 0, 11, 0, 13, 1, 14, 2, 15]","[0, 13, 1, 14, 2, 15, 4, 16, 5, 16]",206,2004
4873,"[6, 8, 12, 15, 17, 21, 0, 2, 6, 8, 11, 15, 17,...","[[[101965.66, 101967.66, 101967.66, 101960.66,...","[17, 7, 18, 8, 20, 8, 21, 9, 22, 10]","[0.28863493, 0.66402495, 0.707597, 0.54336387,...","[20, 8, 21, 9, 23, 10, 0, 11, 0, 13]","[-0.2131495, 0.3065528, -0.0371213, 0.3065528,...","[0, 11, 0, 13, 1, 14, 2, 15, 4, 16]","[1, 14, 2, 15, 4, 16, 5, 16, 5, 17]",207,2004
4874,"[8, 12, 15, 17, 21, 0, 2, 6, 8, 11, 15, 17, 21...","[[[101965.76, 101953.76, 101940.76, 101924.76,...","[20, 8, 21, 9, 23, 10, 0, 11, 0, 13]","[-0.42862818, -0.3951112, -0.52582735, -0.1839...","[23, 12, 0, 13, 0, 14, 1, 14, 2, 15]","[1.2872814, 1.7566899, 1.2537521, 1.7818367, 1...","[1, 14, 1, 14, 2, 15, 2, 16, 4, 16]","[2, 15, 3, 16, 4, 16, 5, 16, 5, 17]",223,2004
...,...,...,...,...,...,...,...,...,...,...
5595,"[8, 11, 15, 17, 21, 0, 2, 6, 8, 12, 15, 17, 21...","[[[101470.0, 101486.0, 101501.0, 101517.0, 101...","[23, 10, 0, 12, 0, 13, 0, 13, 0, 14]","[1.9376696, 1.6092033, 0.86177504, 1.2974956, ...","[0, 13, 1, 14, 1, 14, 2, 15, 2, 15]","[0.021554768, 0.62508005, 0.39037576, 0.457434...","[1, 15, 1, 15, 2, 16, 4, 16, 5, 16]","[3, 16, 4, 16, 5, 16, 5, 17, 6, 18]",292,2010
5596,"[8, 12, 15, 17, 21, 0, 2, 6, 8, 11, 15, 17, 21...","[[[101663.87, 101650.87, 101641.87, 101635.87,...","[0, 12, 0, 13, 0, 13, 0, 14, 1, 15]","[0.86177504, 1.2974956, 0.50984687, 0.87518185...","[1, 14, 1, 14, 2, 15, 2, 15, 3, 16]","[0.39037576, 0.45743412, 0.28140593, 0.4322872...","[1, 15, 2, 16, 4, 16, 5, 16, 5, 17]","[4, 16, 5, 16, 5, 17, 6, 18, 7, 20]",293,2010
5597,"[8, 11, 15, 17, 21, 0, 2, 6, 8, 12, 15, 17, 21...","[[[101876.15, 101838.15, 101801.15, 101765.15,...","[0, 13, 0, 13, 0, 14, 1, 15, 1, 15]","[0.50984687, 0.87518185, 1.8505255, 1.193593, ...","[1, 14, 2, 15, 2, 15, 3, 16, 4, 16]","[0.28140593, 0.43228725, 0.15567149, 0.2478767...","[2, 16, 4, 16, 5, 16, 5, 17, 6, 17]","[5, 16, 5, 17, 6, 18, 7, 20, 8, 21]",294,2010


In [ ]:
Y_train = Y_train.tail(365*2)

## Encoded Image PCA

In [ ]:
pca = PCA(n_components=10)

# Define a function to apply PCA to a single image
def apply_pca(image):
    flat_image = image.reshape(-1, image.shape[-1])
    pca.fit(flat_image)
    return pca.transform(flat_image)

# Apply the PCA model to each image in the dataframe
df_X_train['slp'] = df_X_train['slp'].apply(lambda images: np.stack([apply_pca(image) for image in images]))

In [ ]:
def flatten_array(arr):
    return np.concatenate(arr, axis=None)

df_X_train['slp'] = df_X_train['slp'].apply(flatten_array)

## Model

In [ ]:
df_X_train

,t_slp,slp,t_surge1_input,surge1_input,t_surge2_input,surge2_input,t_surge1_output,t_surge2_output,date,year
id_sequence,,,,,,,,,,
4870,"[2, 6, 8, 11, 15, 17, 21, 0, 2, 6, 8, 12, 15, ...","[-789.01825, 1889.6083, 1119.0668, 294.01807, ...","[16, 5, 16, 5, 16, 6, 17, 7, 18, 8]","[-0.16049244, 0.25846964, -0.09010681, -0.1705...","[17, 6, 18, 7, 18, 8, 20, 8, 21, 9]","[0.40714034, 1.0022833, 0.45743412, 0.42390493...","[20, 8, 21, 9, 22, 10, 0, 11, 0, 13]","[23, 10, 0, 11, 0, 13, 1, 14, 2, 15]",204,2004
4871,"[6, 8, 12, 15, 17, 21, 0, 2, 6, 8, 11, 15, 17,...","[-286.52744, 1486.2375, 694.6795, 407.70175, -...","[16, 5, 16, 6, 17, 7, 18, 8, 20, 8]","[-0.09010681, -0.17054753, 0.27187642, 0.44281...","[18, 7, 18, 8, 20, 8, 21, 9, 23, 10]","[0.45743412, 0.42390493, -0.0035921182, 0.2646...","[21, 9, 22, 10, 0, 11, 0, 13, 1, 14]","[0, 11, 0, 13, 1, 14, 2, 15, 4, 16]",205,2004
4872,"[6, 8, 11, 15, 17, 21, 0, 2, 6, 8, 12, 15, 17,...","[687.01495, -28.784641, 1007.8118, 232.87721, ...","[16, 6, 17, 7, 18, 8, 20, 8, 21, 9]","[0.27187642, 0.44281295, 0.28863493, 0.6640249...","[18, 8, 20, 8, 21, 9, 23, 10, 0, 11]","[-0.0035921182, 0.2646413, -0.2131495, 0.30655...","[22, 10, 0, 11, 0, 13, 1, 14, 2, 15]","[0, 13, 1, 14, 2, 15, 4, 16, 5, 16]",206,2004
4873,"[6, 8, 12, 15, 17, 21, 0, 2, 6, 8, 11, 15, 17,...","[599.0209, -427.6513, -809.90533, 639.61945, -...","[17, 7, 18, 8, 20, 8, 21, 9, 22, 10]","[0.28863493, 0.66402495, 0.707597, 0.54336387,...","[20, 8, 21, 9, 23, 10, 0, 11, 0, 13]","[-0.2131495, 0.3065528, -0.0371213, 0.3065528,...","[0, 11, 0, 13, 1, 14, 2, 15, 4, 16]","[1, 14, 2, 15, 4, 16, 5, 16, 5, 17]",207,2004
4874,"[8, 12, 15, 17, 21, 0, 2, 6, 8, 11, 15, 17, 21...","[1276.5228, 3237.9846, 744.85175, 165.3433, 10...","[20, 8, 21, 9, 23, 10, 0, 11, 0, 13]","[-0.42862818, -0.3951112, -0.52582735, -0.1839...","[23, 12, 0, 13, 0, 14, 1, 14, 2, 15]","[1.2872814, 1.7566899, 1.2537521, 1.7818367, 1...","[1, 14, 1, 14, 2, 15, 2, 16, 4, 16]","[2, 15, 3, 16, 4, 16, 5, 16, 5, 17]",223,2004
...,...,...,...,...,...,...,...,...,...,...
5595,"[8, 11, 15, 17, 21, 0, 2, 6, 8, 12, 15, 17, 21...","[-3251.5547, -2042.1825, 1316.9608, -80.64479,...","[23, 10, 0, 12, 0, 13, 0, 13, 0, 14]","[1.9376696, 1.6092033, 0.86177504, 1.2974956, ...","[0, 13, 1, 14, 1, 14, 2, 15, 2, 15]","[0.021554768, 0.62508005, 0.39037576, 0.457434...","[1, 15, 1, 15, 2, 16, 4, 16, 5, 16]","[3, 16, 4, 16, 5, 16, 5, 17, 6, 18]",292,2010
5596,"[8, 12, 15, 17, 21, 0, 2, 6, 8, 11, 15, 17, 21...","[448.20074, 2908.0603, 386.45914, -532.1743, 3...","[0, 12, 0, 13, 0, 13, 0, 14, 1, 15]","[0.86177504, 1.2974956, 0.50984687, 0.87518185...","[1, 14, 1, 14, 2, 15, 2, 15, 3, 16]","[0.39037576, 0.45743412, 0.28140593, 0.4322872...","[1, 15, 2, 16, 4, 16, 5, 16, 5, 17]","[4, 16, 5, 16, 5, 17, 6, 18, 7, 20]",293,2010
5597,"[8, 11, 15, 17, 21, 0, 2, 6, 8, 12, 15, 17, 21...","[-783.8985, -1215.7332, -562.2615, 505.98303, ...","[0, 13, 0, 13, 0, 14, 1, 15, 1, 15]","[0.50984687, 0.87518185, 1.8505255, 1.193593, ...","[1, 14, 2, 15, 2, 15, 3, 16, 4, 16]","[0.28140593, 0.43228725, 0.15567149, 0.2478767...","[2, 16, 4, 16, 5, 16, 5, 17, 6, 17]","[5, 16, 5, 17, 6, 18, 7, 20, 8, 21]",294,2010


In [ ]:
t_slp = df_X_train['t_slp'].values
slp = df_X_train['slp'].values
t_surge1_input = df_X_train['t_surge1_input'].values
surge1_input = df_X_train['surge1_input'].values
t_surge2_input = df_X_train['t_surge2_input'].values
surge2_input = df_X_train['surge2_input'].values
t_surge1_output = df_X_train['t_surge1_output'].values
t_surge2_output = df_X_train['t_surge2_output'].values
date = df_X_train['date'].values
year = df_X_train['year'].values

X = np.vstack([np.hstack(row) for row in df_X_train.to_numpy()])
y = Y_train.to_numpy()

In [ ]:
len(X)

730

In [ ]:
# create XGBoost DMatrix
dtrain1 = xgb.DMatrix(X, label=y)

# set XGBoost parameters
params1 = {'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'seed': 42, 'verbosity':2}

# perform cross-validation
cv_results = xgb.cv(params=params1, dtrain=dtrain1, num_boost_round=2, nfold=5, early_stopping_rounds=10)

# print the last evaluation metrics for each cross-validation round
print(cv_results.tail())

In [ ]:
# new attempt 9pm 5/13
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# Define the cross-validation object
kf = KFold(n_splits=5, shuffle=True)

# Define the XGBoost model and its hyperparameters
params = {'max_depth': 3, 'eta': 0.1, 'objective': 'reg:squarederror','verbosity':2}
num_round = 10

# Train the model on each training set and evaluate its performance on the corresponding test set
scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    model = xgb.train(params, dtrain, num_round)
    preds = model.predict(dtest)
    score = mean_squared_error(y_test, preds, squared=False)
    scores.append(score)

# Calculate the average performance of the model across all the folds
average_score = np.mean(scores)

# Train the model on the entire dataset using the optimal hyperparameters
dtrain_all = xgb.DMatrix(X, label=y)
model = xgb.train(params, dtrain_all, num_round)

In [ ]:
print(scores)
print(average_score)

[0.765970264931737, 0.8018566464188541, 0.7081001172135963, 0.7610166017726304, 0.7657475653962901]
0.7605382391466217


In [ ]:
print(model)

In [ ]:
feature_scores = model.get_score(importance_type='weight')
print(feature_scores)

{'f0': 9.0, 'f1': 3.0, 'f2': 1.0, 'f3': 6.0, 'f5': 5.0, 'f6': 2.0, 'f14': 1.0, 'f15': 4.0, 'f40': 5.0, 'f41': 2.0, 'f42': 3.0, 'f43': 4.0, 'f44': 1.0, 'f45': 1.0, 'f46': 3.0, 'f47': 1.0, 'f48': 1.0, 'f49': 1.0, 'f52': 1.0, 'f55': 1.0, 'f56': 1.0, 'f57': 1.0, 'f59': 1.0, 'f61': 1.0, 'f62': 2.0, 'f64': 1.0, 'f65': 1.0, 'f67': 2.0, 'f68': 1.0, 'f71': 2.0, 'f73': 1.0, 'f75': 1.0, 'f77': 1.0, 'f79': 1.0, 'f84': 1.0, 'f89': 1.0, 'f94': 1.0, 'f103': 1.0, 'f104': 1.0, 'f107': 1.0, 'f115': 1.0, 'f118': 1.0, 'f122': 1.0, 'f125': 2.0, 'f131': 1.0, 'f138': 1.0, 'f141': 1.0, 'f146': 1.0, 'f157': 1.0, 'f162': 1.0, 'f166': 1.0, 'f180': 1.0, 'f187': 1.0, 'f210': 1.0, 'f215': 1.0, 'f222': 1.0, 'f225': 1.0, 'f227': 1.0, 'f229': 1.0, 'f230': 1.0, 'f253': 1.0, 'f275': 1.0, 'f276': 2.0, 'f282': 1.0, 'f313': 1.0, 'f328': 1.0, 'f332': 2.0, 'f351': 1.0, 'f363': 1.0, 'f384': 1.0, 'f387': 1.0, 'f391': 1.0, 'f399': 1.0, 'f443': 1.0, 'f451': 1.0, 'f475': 1.0, 'f478': 1.0, 'f535': 1.0, 'f537': 1.0, 'f539': 1.0, 'f

In [ ]:
# Define XGBoost parameters
params = {
    'objective': 'reg:squarederror',  # Use regression objective
    'n_estimators': 1000,            # Number of trees in the forest
    'max_depth': 5,                  # Maximum depth of each tree
    'learning_rate': 0.1,            # Learning rate for gradient descent
    'subsample': 0.8,                # Fraction of training data to use for each tree
    'colsample_bytree': 0.8,         # Fraction of features to use for each tree
    'reg_alpha': 0.1,                # L1 regularization parameter
    'reg_lambda': 0.1,               # L2 regularization parameter
    'seed': 123                      # Random seed for reproducibility
}

# Train XGBoost model for output variable 1
dtrain1 = xgb.DMatrix(X, label=y1)
model1 = xgb.train(params, dtrain1)

# Train XGBoost model for output variable 2
dtrain2 = xgb.DMatrix(X, label=y2)
model2 = xgb.train(params, dtrain2)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import time

def find_closest_sequences(df, idx, k=5):
    start_time = time.time()
    # Extract the sequence to compare
    seq = df.iloc[idx]["slp"]
    
    # Compute pairwise cosine similarity between each image in the sequence and all images in all sequences in the DataFrame
    similarities = []
    for img in seq:
        sim = cosine_similarity(np.vstack([img.flatten() for img in df["slp"].values]))
        similarities.append(sim.flatten())
    similarities = np.array(similarities)
    
    # Compute the average cosine similarity for each sequence
    avg_similarities = np.mean(similarities, axis=0)
    
    # Get the top k indices of most similar sequences
    top_k_indices = np.argsort(avg_similarities)[::-1][:k]
    
    # Return the closest sequences as a list of tuples (index, similarity)
    closest_sequences = [(i, avg_similarities[i]) for i in top_k_indices]
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    print(closest_sequences)
    return closest_sequences

In [ ]:
closest_sequences = find_closest_sequences(df_X_train, 1200, k=5)

Elapsed time: 242.96 seconds


In [ ]:
print(closest_sequences)

NameError: ignored

In [ ]:
def dtw_distance(s1, s2):
    n, m = len(s1), len(s2)
    dtw = np.zeros((n+1, m+1))
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = np.linalg.norm(s1[i-1] - s2[j-1])
            dtw[i][j] = cost + min(dtw[i-1][j], dtw[i][j-1], dtw[i-1][j-1])
    return dtw[n][m]

def find_closest_sequences(df, idx, k=None):
    # Extract the sequence to compare
    seq = df.iloc[idx]["slp"]
    
    # Compute DTW distance between the sequence and all prior sequences in the DataFrame
    distances = []
    for i in range(idx):
        dist = dtw_distance(seq, df.iloc[i]["slp"])
        distances.append(dist)
    distances = np.array(distances)
    
    # Get the top k indices of most similar sequences
    if k is None:
        k = len(distances)
    else:
        k = min(k, len(distances))
    top_k_indices = np.argsort(distances)[:k]
    
    # Return the closest sequences as a list of tuples (index, distance)
    closest_sequences = [(i, distances[i]) for i in top_k_indices]
    print(closest_sequences)
    return closest_sequences

In [ ]:
closestsequences1 = find_closest_sequences(df_X_train,1200,k=500)

[(1200, 0.0), (1201, 136086.3330078125), (1199, 189021.2705078125), (1202, 408961.6142578125), (1203, 717706.0830078125), (1204, 977736.9404296875), (3862, 983634.537109375), (4124, 988676.796875), (3128, 990677.9375), (860, 993487.4375), (1445, 999618.01953125), (3129, 1011834.455078125), (3142, 1014400.505859375), (3127, 1023757.671875), (92, 1024090.609375), (1444, 1026640.162109375), (2336, 1027172.720703125), (5175, 1034234.048828125), (3671, 1036943.22265625), (1330, 1038159.64453125), (2622, 1038413.724609375), (3861, 1042137.302734375), (5177, 1042928.76953125), (2335, 1043333.306640625), (2517, 1044991.943359375), (3899, 1047431.638671875), (3126, 1047772.396484375), (3125, 1050617.693359375), (5173, 1052374.1484375), (3863, 1061500.12109375), (5174, 1062094.162109375), (338, 1064966.0390625), (2370, 1068319.841796875), (859, 1069851.32421875), (5194, 1074700.65234375), (5516, 1079411.912109375), (1867, 1079997.7265625), (861, 1081412.40625), (1637, 1082751.486328125), (1051, 

In [ ]:
closestsequences2 = find_closest_sequences(df_X_train,3000,k=500)

[(3000, 0.0), (2999, 99032.67041015625), (3001, 120420.61474609375), (2998, 333260.4052734375), (3002, 392193.17919921875), (4183, 734269.8486328125), (4182, 739938.572265625), (4181, 765991.0068359375), (203, 790386.4423828125), (202, 803547.5869140625), (2775, 816079.537109375), (2916, 816468.732421875), (2917, 854769.115234375), (4184, 876882.9736328125), (3731, 888407.298828125), (4300, 889780.0224609375), (2658, 889983.369140625), (752, 903611.892578125), (2997, 905587.5888671875), (2774, 907711.283203125), (204, 915727.7998046875), (282, 915815.98046875), (4433, 917699.861328125), (2657, 921675.642578125), (3931, 924049.455078125), (3732, 937303.958984375), (3505, 939976.2373046875), (283, 941389.6884765625), (281, 943805.833984375), (751, 948893.259765625), (707, 950824.9697265625), (298, 953612.62109375), (2918, 953622.912109375), (201, 953668.021484375), (2776, 953827.625), (299, 953916.2958984375), (3403, 954590.25390625), (4434, 955758.904296875), (2773, 956213.818359375), (

In [ ]:
closestsequences3 = find_closest_sequences(df_X_train,2400,k=500)

[(2400, 0.0), (2401, 153449.89453125), (2399, 161289.10498046875), (2402, 390143.603515625), (2398, 393277.37255859375), (58, 656135.30859375), (2403, 668782.556640625), (59, 685319.404296875), (2892, 687078.421875), (296, 708938.033203125), (4318, 720289.23046875), (2890, 726443.2890625), (57, 728314.0263671875), (2538, 730178.9521484375), (2539, 736482.6650390625), (2085, 739259.53125), (5401, 740989.3916015625), (508, 745492.2578125), (1648, 745903.0009765625), (1039, 749523.697265625), (1035, 753338.5732421875), (207, 754657.619140625), (2891, 759830.6494140625), (4317, 761734.2373046875), (1901, 762298.029296875), (1040, 763588.1171875), (1041, 764457.2880859375), (439, 765121.623046875), (5060, 766205.052734375), (509, 766935.2001953125), (1983, 768455.4892578125), (5573, 769249.4033203125), (295, 770627.275390625), (60, 771766.2431640625), (1107, 772140.2158203125), (1126, 774047.3994140625), (5570, 778258.5458984375), (1036, 778280.7001953125), (4740, 778287.19140625), (2562, 7

In [ ]:
closestsequences4 = find_closest_sequences(df_X_train,2600,k=500)

[(2600, 0.0), (2599, 208148.4150390625), (2601, 276220.806640625), (2598, 485321.4736328125), (2602, 531893.498046875), (2603, 801782.275390625), (2870, 963797.40625), (453, 1012784.4296875), (2869, 1017869.177734375), (456, 1032040.5390625), (452, 1034542.404296875), (2453, 1039699.17578125), (3152, 1052722.2529296875), (3151, 1054000.4130859375), (1906, 1058171.0673828125), (455, 1079840.98046875), (612, 1084117.388671875), (454, 1092179.14453125), (560, 1095113.806640625), (611, 1095794.01953125), (1905, 1109252.037109375), (559, 1117548.0625), (539, 1120797.498046875), (3153, 1124223.0947265625), (5036, 1128866.611328125), (3154, 1135097.7841796875), (1934, 1143332.4248046875), (2868, 1146790.57421875), (451, 1147026.5400390625), (613, 1150984.5234375), (1443, 1156111.326171875), (847, 1163484.205078125), (4563, 1168545.228515625), (609, 1169340.3984375), (3070, 1170442.748046875), (1442, 1170782.759765625), (1564, 1175955.4287109375), (1565, 1178042.5576171875), (2454, 1180603.111